In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.shape

(42000, 785)

In [4]:
test.shape

(28000, 784)

In [5]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X = train.drop(['label'],axis=1)
y = pd.get_dummies(train['label'])

In [7]:
X = X.values
y = y.values
X = X.reshape(X.shape[0],1,28,28)

In [8]:
X = X/255.0
from sklearn.cross_validation import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)

In [24]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import Adam

model = Sequential()

model.add(Convolution2D(4, 3, 3, border_mode='same', input_shape=(1, 28, 28),init='lecun_uniform'))
model.add(Activation('relu'))
model.add(Convolution2D(2, 1, 1, init='lecun_uniform',border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(2, 3, 1, init='lecun_uniform',border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(2, 1, 3, init='lecun_uniform',border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(4, 1, 1, init='lecun_uniform',border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(8, 3, 3, border_mode='same',init='lecun_uniform'))
model.add(Activation('relu'))
model.add(Convolution2D(4, 1, 1 , init='lecun_uniform', border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(4, 3, 1 , init='lecun_uniform', border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(4, 1, 3 , init='lecun_uniform', border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(8, 1, 1 , init='lecun_uniform', border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
# Note: Keras does automatic shape inference.
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(10))
model.add(Activation('softmax'))


In [31]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=["accuracy"])
model.summary()

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
convolution2d_46 (Convolution2D)   (None, 4, 28, 28)   40          convolution2d_input_7[0][0]      
____________________________________________________________________________________________________
activation_58 (Activation)         (None, 4, 28, 28)   0           convolution2d_46[0][0]           
____________________________________________________________________________________________________
convolution2d_47 (Convolution2D)   (None, 2, 28, 28)   10          activation_58[0][0]              
____________________________________________________________________________________________________
activation_59 (Activation)         (None, 2, 28, 28)   0           convolution2d_47[0][0]           
___________________________________________________________________________________________

In [39]:
model.fit(X, y, batch_size=256, nb_epoch=2)

Epoch 1/2
42000/42000 [==============================] - 29s - loss: 0.0365 - acc: 0.9875    
Epoch 2/2
42000/42000 [==============================] - 28s - loss: 0.0366 - acc: 0.9873    


In [43]:
test_data = test.values
test_data = test_data / 255.0
test_data = test_data.reshape(test_data.shape[0],1,28,28)
yPred = model.predict_classes(test_data)

28000/28000 [==============================] - 7s     


In [44]:
columns = ['ImageId','Label']
df = pd.DataFrame(data=np.zeros((test.shape[0],2)), columns=columns)
df['ImageId'] = xrange(test.shape[0])
df['ImageId'] += 1
df['Label'] = yPred
df.head(10)

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
5,6,7
6,7,0
7,8,3
8,9,0
9,10,3


In [45]:
df.to_csv('CNN_keras.csv',index=False)

Got 0.98614 on Kaggle with roughly 30min training